In [1]:
import os
os.chdir('../')

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    params_image_size : list
    params_batch_size: int

In [8]:
from cnnClassfier.constants import *
from cnnClassfier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAZE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [15]:
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {'loss' : self.score[0], 'accuracy' : self.score[1]}
        save_json(path = Path('scores.json'), data = scores)

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e

[2024-07-28 02:01:54,885: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-28 02:01:54,889: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-28 02:01:54,890: INFO: common: Created directory  at: artifacts]
Found 116 images belonging to 2 classes.

[2024-07-28 02:01:56,004: WARNING: module_wrapper: From c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
8/8 [==============================] - 11s 1s/step - loss: 0.3306 - accuracy: 0.9569
[2024-07-28 02:02:06,982: INFO: common: Json file saved at: scores.json]
